## Imports

In [1]:
from typing import Callable, List
import tensorflow_hub as hub
import tensorflow_text as tft
import tensorflow as tf
import wandb

import matplotlib.pyplot as plt
import numpy as np
import random
import time

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## Contants

In [2]:
TFRECORDS_DIR = "gs://variable-length-sequences-tf/tfrecords"
BERT_MAX_SEQLEN = 512
BATCH_SIZE = 64

## TFRecord parsing utilities

In [3]:
feature_descriptions = {
    "summary": tf.io.FixedLenFeature([], dtype=tf.string),
    "summary_tokens": tf.io.FixedLenFeature([], dtype=tf.string),
    "summary_tokens_len": tf.io.FixedLenFeature([1], dtype=tf.int64),
    "label": tf.io.FixedLenFeature([1], dtype=tf.int64),
}

In [4]:
def deserialize_composite(serialized, type_spec):
    """Parses a serialized Ragged features and retains the original structure."""
    serialized = tf.io.parse_tensor(serialized, tf.string)
    component_specs = tf.nest.flatten(type_spec, expand_composites=True)
    components = [
        tf.io.parse_tensor(serialized[i], spec.dtype)
        for i, spec in enumerate(component_specs)
    ]
    return tf.nest.pack_sequence_as(type_spec, components, expand_composites=True)


def read_example(example):
    """Parses a single TFRecord file."""
    features = tf.io.parse_single_example(example, feature_descriptions)
    features["summary_tokens"] = deserialize_composite(
        features.get("summary_tokens"),
        tf.RaggedTensorSpec(dtype=tf.int32, ragged_rank=2),
    )

    return features

In [5]:
def set_text_preprocessor(preprocessor_path: str) -> Callable:
    """ Decorator to set the desired preprocessor for a
        function from a TensorFlow Hub URL.
        
    Arguments:
        preprocessor_path {str} -- URL of the TF-Hub preprocessor.
    
    Returns:
        Callable -- A function with the `preprocessor` attribute set.
    """
    def decoration(func: Callable):
        # Loading the preprocessor from TF-Hub
        preprocessor = hub.load(preprocessor_path)
        
        # Setting an attribute called `preprocessor` to
        # the passed function
        func.preprocessor = preprocessor
        return func
    return decoration

In [6]:
@set_text_preprocessor(
    preprocessor_path="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
def preprocess_variable_batch(batch):
    """Batch processing utility."""
    text_tokens_max_len = tf.cast(
        tf.math.reduce_max(batch["summary_tokens_len"]), dtype=tf.int32,
    )

    # Generating the inputs for the BERT model.
    bert_input_packer = hub.KerasLayer(
        preprocess_variable_batch.preprocessor.bert_pack_inputs,
        arguments={"seq_length": tf.minimum(text_tokens_max_len, BERT_MAX_SEQLEN)},
    )
    bert_packed_text = bert_input_packer(
        [tf.squeeze(batch.pop("summary_tokens"), axis=1)]
    )

    labels = batch.pop("label")
    return bert_packed_text, labels

2021-12-17 14:32:46.661962: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-17 14:32:52.523054: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38414 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


In [31]:
# Find the longest sequence in the training set
ds = tf.data.Dataset.list_files(f"{TFRECORDS_DIR}/train-*.tfrecord")
ds = tf.data.TFRecordDataset(ds).map(
    lambda example: tf.io.parse_single_example(
        example, feature_descriptions
    )
)
max_seq_len = tf.cast(
    tf.reduce_max([datum["summary_tokens_len"] for datum in ds]), tf.int32)
print(f"Longest token sequence in the training split: {max_seq_len.numpy()}")

Longest token sequence in the training split: 2947


In [32]:
preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
fixed_len_bert_packer = hub.KerasLayer(
    preprocessor.bert_pack_inputs,
    arguments={
        "seq_length": tf.minimum(max_seq_len,  BERT_MAX_SEQLEN)
    }
)


In [33]:
def preprocess_fixed_batch(batch):
    """Batch processing utility."""

    # Generating the inputs for the BERT model.
    bert_packed_text = fixed_len_bert_packer(
        [tf.squeeze(batch.pop("summary_tokens"), axis=1)]
    )

    labels = batch.pop("label")
    return bert_packed_text, labels

## Dataset preparation

In [8]:
def get_dataset(
    split: str,
    batch_size: int,
    batch_preprocessor: Callable,
    shuffle: bool
):
    """Prepares tf.data.Dataset objects from TFRecords."""
    ds = tf.data.Dataset.list_files(f"{TFRECORDS_DIR}/{split}-*.tfrecord")
    ds = ds.interleave(
        tf.data.TFRecordDataset,
        cycle_length=tf.data.AUTOTUNE,
        num_parallel_calls=tf.data.AUTOTUNE,
    ).map(
        read_example,
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=False
    ).cache()
    if shuffle:
        ds = ds.shuffle(batch_size * 10)
    ds = ds.batch(
        batch_size
    ).map(
        batch_preprocessor,
        num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)
    return ds

## Analyzing the maximum sequence lengths over training batches

In [9]:
# analysis_ds = tf.data.Dataset.list_files(f"{TFRECORDS_DIR}/train-*.tfrecord")
# analysis_ds = analysis_ds.interleave(
#     tf.data.TFRecordDataset, cycle_length=3, num_parallel_calls=tf.data.AUTOTUNE
# )
# analysis_ds = analysis_ds.map(
#     read_example, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False
# )
# analysis_ds = analysis_ds.batch(BATCH_SIZE)

In [10]:
# max_seqlens = []
# batches = 0


# for batch in analysis_ds:
#     max_seqlens.append(
#         int(tf.cast(tf.math.reduce_max(batch["summary_tokens_len"]), dtype=tf.int32,))
#     )
#     batches += 1

# plt.plot(np.arange(batches), max_seqlens)
# plt.xlabel("Batch #", fontsize=14)
# plt.ylabel("Maximum sequence lengths", fontsize=14)
# plt.show()

## Model Building

In [11]:
def genre_classifier(
    encoder_path: str,
    input_features: List[str],
    train_encoder: bool,
    proj_dim: int,
    num_labels: int
):
    """Creates a simple classification model."""
    sentence_encoder = hub.KerasLayer(encoder_path)
    sentence_encoder.trainable = train_encoder

    inputs = {
        feature_name: tf.keras.Input(
            shape=(None,), dtype=tf.int32, name=feature_name
        )
        for feature_name in input_features
    }

    text_encodings = sentence_encoder(inputs)
    projections = tf.keras.layers.Dense(proj_dim, activation="relu")(text_encodings["pooled_output"])
    probs = tf.keras.layers.Dense(num_labels, activation="softmax")(projections)
    return tf.keras.Model(inputs=inputs, outputs=probs)

In [12]:
cmlm_uri = "https://tfhub.dev/google/universal-sentence-encoder-cmlm/en-base/1"
cmlm_inputs = ["input_word_ids", "input_type_ids", "input_mask"]
proj_dim = 128
num_labels = 27

num_epochs = 5
num_runs = 10

## Training with variable batch width

In [13]:
train_ds = get_dataset("train", BATCH_SIZE, preprocess_variable_batch, True)
valid_ds = get_dataset("val", BATCH_SIZE, preprocess_variable_batch, False)
test_ds = get_dataset("test", BATCH_SIZE, preprocess_variable_batch, False)

In [14]:
for i in range(num_runs):
    wandb.init(
        project="batching-experiments",
        entity="carted",
        name=f"variable-legth-run:{i + 1}",
        group="variable-length-batching",
    )
    model = genre_classifier(cmlm_uri, cmlm_inputs, False, proj_dim, num_labels)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")
    start = time.time()
    model.fit(
        train_ds,
        epochs=5,
        validation_data=valid_ds,
        callbacks=[wandb.keras.WandbCallback()]
    )
    end = time.time()
    wandb.log({"model_training_time (seconds)": end - start})
    
    loss, acc = model.evaluate(test_ds)
    wandb.log({"test_loss": loss})
    wandb.log({"test_acc": acc})
    
    wandb.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carted (use `wandb login --relogin` to force relogin)
/opt/conda/envs/var-len-text/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


Epoch 1/5


2021-12-17 14:35:08.112230: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


763/763 [==============================] - 227s 275ms/step - loss: 1.5181 - accuracy: 0.5638 - val_loss: 1.2519 - val_accuracy: 0.6125
Epoch 2/5
763/763 [==============================] - 211s 276ms/step - loss: 1.1924 - accuracy: 0.6284 - val_loss: 1.1885 - val_accuracy: 0.6276
Epoch 3/5
763/763 [==============================] - 210s 275ms/step - loss: 1.1314 - accuracy: 0.6425 - val_loss: 1.1626 - val_accuracy: 0.6286
Epoch 4/5
763/763 [==============================] - 210s 276ms/step - loss: 1.0935 - accuracy: 0.6509 - val_loss: 1.1617 - val_accuracy: 0.6337
Epoch 5/5
847/847 [==============================] - 207s 243ms/step - loss: 1.1696 - accuracy: 0.6295


accuracy,▁▆▇▇█
epoch,▁▃▅▆█
loss,█▃▂▁▁
model_training_time (seconds),▁
test_acc,▁
test_loss,▁
val_accuracy,▁▆▆█▇
val_loss,█▃▁▁▁
accuracy,0.65904
best_epoch,3
best_val_loss,1.16173


## Training with fixed length batch width

In [34]:
train_ds = get_dataset("train", BATCH_SIZE, preprocess_fixed_batch, True)
valid_ds = get_dataset("val", BATCH_SIZE, preprocess_fixed_batch, False)
test_ds = get_dataset("test", BATCH_SIZE, preprocess_fixed_batch, False)

In [ ]:
for i in range(num_runs):
    wandb.init(
        project="batching-experiments",
        entity="carted",
        name=f"fixed-legth-run:{i + 1}",
        group="fixed-length-batching",
    )
    model = genre_classifier(cmlm_uri, cmlm_inputs, False, proj_dim, num_labels)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics="accuracy")
    start = time.time()
    model.fit(
        train_ds,
        epochs=5,
        validation_data=valid_ds,
        callbacks=[wandb.keras.WandbCallback()]
    )
    end = time.time()
    wandb.log({"model_training_time (seconds)": end - start})
    
    loss, acc = model.evaluate(test_ds)
    wandb.log({"test_loss": loss})
    wandb.log({"test_acc": acc})
    
    wandb.finish()

Epoch 1/5
763/763 [==============================] - 252s 310ms/step - loss: 1.5073 - accuracy: 0.5637 - val_loss: 1.2386 - val_accuracy: 0.6175
Epoch 2/5
160/763 [=====>........................] - ETA: 2:46 - loss: 1.1884 - accuracy: 0.6332

In [ ]:
!sudo shutdown now